In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.nn.utils.rnn import pad_sequence

from utils.mails import send_mail
from utils.checkpoints import save_checkpoint, load_checkpoint



# Hyper Parameter 세팅

In [3]:
# dataset path
DATA_DF_PATH = './dataset/'
DATA_IMG_PATH = '../dataset/kaggle_rsna(only100)/'

# hyper parameter
MODEL_NAME = 'SequentialGRU'
LOSS_NAME = 'BCE'
EPOCHS = 100
BATCH_SIZE = 4
INITIAL_LR = 0.001

# etc path
MODEL_WEIGHTS_SAVE_PATH = './checkpoints/rnn'
MODEL_WEIGHTS_LOAD_PATH = './checkpoints/rnn'
TENSORBOARD_PATH = './tensorboard/rnn'

# gpu settings
IS_GPU_PARALLEL = True if torch.cuda.device_count()>1 else False

# train id
timestamp = datetime.now().strftime('%y%m%d_%H%M%S')
TRAIN_ID = f'{timestamp}_{MODEL_NAME}_LR{INITIAL_LR}_BS{BATCH_SIZE}_{LOSS_NAME}Loss'

# 데이터셋 구성

In [4]:
class SequentialHmData(Dataset):
    def __init__(self, feature_path, df_path):
        self.feature_df = pd.read_csv(feature_path)
        self.ref_df = pd.read_csv(df_path)
        self.person_ids = self.ref_df.study_instance_uid.unique()
        
    def __getitem__(self, index):
        # get patient id
        current_person_id = self.person_ids[index]
        
        # get filenames corresponding with patient id
        filenames = self.ref_df[self.ref_df.study_instance_uid==current_person_id].filename
        df_current_person = self.feature_df[self.feature_df.filename.isin(filenames)]
        
        # get predicted label and features from cnn outputs
        pred_label = df_current_person.iloc[:,1:7].values
        pred_features = df_current_person.iloc[:,7:].values
        
        # get get label
        gt_label = self.ref_df[self.ref_df.study_instance_uid==current_person_id].loc[:,'epidural':'any'].values
        
        
        return torch.from_numpy(pred_label), torch.from_numpy(pred_features), torch.from_numpy(gt_label)
    
    def __len__(self):
        return len(self.ref_df.study_instance_uid.unique())

In [5]:
train_dataset = SequentialHmData(feature_path='./dataset/train_features.csv', df_path='./dataset/train.csv')
valid_dataset = SequentialHmData(feature_path='./dataset/valid_features.csv', df_path='./dataset/valid.csv')
test_dataset = SequentialHmData(feature_path='./dataset/test_features.csv', df_path='./dataset/test.csv')

In [6]:
from torch.utils.data import DataLoader

def make_pad_sequence(datas):
    
    pred_labels = [data[0] for data in datas]
    pred_features = [data[1] for data in datas]
    gt_labels = [data[2] for data in datas]
    
    # pad all sequence 
    pred_labels = pad_sequence(pred_labels, batch_first=True)
    pred_features = pad_sequence(pred_features, batch_first=True)
    gt_labels = pad_sequence(gt_labels, batch_first=True)

    # shape: (N, SL, F) -> (N, SL, 1, F) -> (N, F, 1, SL) 
    pred_labels = pred_labels.unsqueeze(dim=2).permute(0,3,2,1)
    pred_features = pred_features.unsqueeze(dim=2).permute(0,3,2,1)
    gt_labels = gt_labels.unsqueeze(dim=2).permute(0,3,2,1)
        
    return pred_labels, pred_features, gt_labels

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=make_pad_sequence)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=make_pad_sequence)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=make_pad_sequence)

In [7]:
for data in train_loader:
    print(data[0].shape, data[1].shape, data[2].shape)
    break

torch.Size([4, 6, 1, 43]) torch.Size([4, 1024, 1, 43]) torch.Size([4, 6, 1, 43])


# 모델 생성

In [19]:
class SequenceModel(nn.Module):
    def __init__(self, ch_in=1024):
        super(SequenceModel, self).__init__()
        drop_out = 0.5
        hidden = 96
        lstm_layers = 2
        feature_num=1
        ratio = 1
        self.ratio=ratio
        
        # seq model 1
        self.fea_conv = nn.Sequential(nn.Dropout2d(drop_out),
                                      nn.Conv2d(ch_in, 512, kernel_size=(1, 1), stride=(1,1),padding=(0,0), bias=False),
                                      nn.BatchNorm2d(512),
                                      nn.ReLU(),
                                      nn.Dropout2d(drop_out),
                                      nn.Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=False),
                                      nn.BatchNorm2d(128),
                                      nn.ReLU(),
                                      nn.Dropout2d(drop_out),
                                      )

        self.fea_first_final = nn.Sequential(nn.Conv2d(128*feature_num, 6, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=True))

        # # bidirectional GRU
        self.hidden_fea = hidden
        self.fea_lstm = nn.GRU(128*feature_num, self.hidden_fea, num_layers=lstm_layers, batch_first=True, bidirectional=True)
        self.fea_lstm_final = nn.Sequential(nn.Conv2d(1, 6, kernel_size=(1, self.hidden_fea*2), stride=(1, 1), padding=(0, 0), dilation=1, bias=True))
        
        
        # seq model 2
        self.conv_first = nn.Sequential(nn.Conv2d(12, 128*ratio, kernel_size=(5, 1), stride=(1,1),padding=(2,0),dilation=1, bias=False),
                                        nn.BatchNorm2d(128*ratio),
                                        nn.ReLU(),
                                        nn.Conv2d(128*ratio, 64*ratio, kernel_size=(3, 1), stride=(1, 1), padding=(2, 0),dilation=2, bias=False),
                                        nn.BatchNorm2d(64*ratio),
                                        nn.ReLU())

        self.conv_res = nn.Sequential(nn.Conv2d(64 * ratio, 64 * ratio, kernel_size=(3, 1), stride=(1, 1),padding=(4, 0),dilation=4, bias=False),
                                      nn.BatchNorm2d(64 * ratio),
                                      nn.ReLU(),
                                      nn.Conv2d(64 * ratio, 64 * ratio, kernel_size=(3, 1), stride=(1, 1),padding=(2, 0),dilation=2, bias=False),
                                      nn.BatchNorm2d(64 * ratio),
                                      nn.ReLU())

        self.conv_final = nn.Sequential(nn.Conv2d(64*ratio, 1, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), dilation=1,bias=False))

        # bidirectional GRU
        self.hidden = hidden
        self.lstm = nn.GRU(64*ratio, self.hidden, num_layers=lstm_layers, batch_first=True, bidirectional=True)
        self.final = nn.Sequential(nn.Conv2d(1, 6, kernel_size=(1, self.hidden*2), stride=(1, 1), padding=(0, 0), dilation=1, bias=True))


    def forward(self, features, x):
        
        batch_size, _, _, _ = features.shape
        
        # stem_fc
        x_fc = self.fea_conv(features) # (N, LenFeat, 1, LenSeq)
        
        # fc
        out11 = self.fea_first_final(x_fc) # (N, 6, 1, LenSeq)

        # lstm
        x_lstm, _ = self.fea_lstm(x_fc.view(batch_size, -1, 128)) # (N, LenSeq,192)
        x_lstm = x_lstm.reshape(batch_size, 1, -1, self.hidden_fea*2) # (N, 1, LenSeq, 192)
        out12 = self.fea_lstm_final(x_lstm) # (N, 6, 1, LenSeq)
        out12 = out12.permute(0,1,3,2)
        
        # seq1 output
        out1 = out11+out12
        out1_sigmoid = torch.sigmoid(out1)
        
        # concat cnn out, seq1 out
        x = torch.cat([x, out1_sigmoid], dim=1)
        
        # stem_fc
        x = self.conv_first(x)
        x = self.conv_res(x)
        
        # fc
        out21 = self.conv_final(x)
        
        # lstm
        x, _ = self.lstm(x.view(batch_size, -1, 64))
        x = x.reshape(batch_size, 1, -1, self.hidden*2)
        out22 = self.final(x)
        out22 = out22.permute(0,1,3,2)
        
        # seq2 output
        out2 = out21 + out22
        out2_sigmoid = torch.sigmoid(out2)
        
        return out1_sigmoid, out2_sigmoid


# 학습

In [20]:
model = SequenceModel(ch_in=1024)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=0.9)

In [21]:
def fit(phase, epoch, model, data_loader, optimizer, criterion, device):
    
    losses = 0
    if phase=='Train':
        model.train()
    elif phase=='Valid' or phase=='Test':
        model.eval()
    
    tbar = tqdm(data_loader, position=0, leave=True)
    for i, (pred_label, pred_feature, gt_label) in enumerate(tbar):
        
        pred_label, pred_feature, gt_label = pred_label.to(device), pred_feature.to(device), gt_label.to(device)
        
        optimizer.zero_grad()
        
        logit1, logit2 = model(pred_feature.float(), pred_label.float())
        
        loss1 = criterion(logit1, gt_label.float())
        loss2 = criterion(logit2, gt_label.float())
        
        loss = loss1 + loss2
        
        if phase=='Train':
            loss.backward()
            optimizer.step()
        
#         predicted_label_thresholded = predicted_label>0.5
#         acc = (predicted_label_thresholded==target).sum() # 

        losses += loss.item()
        
        tbar.set_description(f'[{phase}]\tEpoch:[{epoch}/{EPOCHS}]\tLoss:{losses/(i+1):.5f}')# '\tAcc:{acc:.2%}')
        
    return losses/len(data_loader)
        

In [22]:
# tensorboard log
writer = SummaryWriter(log_dir=os.path.join(TENSORBOARD_PATH, TRAIN_ID))
train_losses = []
valid_losses = []

for epoch in range(1, EPOCHS+1):
        
    # fit
    train_loss = fit('Train', epoch, model, train_loader, optimizer, criterion, device)
    with torch.no_grad():
        valid_loss = fit('Valid', epoch, model, valid_loader, optimizer, criterion, device)
        
    # log
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    # tensor board
    writer.add_scalar('Loss/Train/', train_loss, epoch)
    writer.add_scalar('Loss/Valid/', valid_loss, epoch)

    # save model
    save_checkpoint(epoch, model, optimizer)
    
#     if epoch%10==0:
#         send_mail(f'[Epoch:{epoch}]학습 진행중', '')
    
# send_mail(f'[알림]학습완료','EC2 종료할 것!!')




[Train]	Epoch:[2/100]	Loss:nan:  70%|███████   | 14/20 [00:00<00:00, 14.26it/s]

KeyboardInterrupt: 

[Train]	Epoch:[2/100]	Loss:nan:  70%|███████   | 14/20 [00:20<00:00, 14.26it/s]

In [12]:
def save_checkpoint(epoch, model, optimizer, scheduler=None):
    save_folder_dir = os.path.join(MODEL_WEIGHTS_SAVE_PATH, TRAIN_ID)
    if not os.path.exists(save_folder_dir):
        os.makedirs(save_folder_dir, exist_ok=True)
    model_save_path = os.path.join(save_folder_dir, f'{epoch:03d}.pth')

    if IS_GPU_PARALLEL:
        model_state_dict = model.module.state_dict()
    else:
        model_state_dict = model.state_dict()

    if scheduler is not None:
        scheduler = scheduler.state_dict()

    torch.save({
        'epoch': epoch,
        'model_state_dict': model_state_dict,
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler
    }, model_save_path)